In [ ]:
!pip install plotly

In [61]:
!pip uninstall -y numpy
!pip install numpy

Found existing installation: numpy 1.16.6
Uninstalling numpy-1.16.6:
  Successfully uninstalled numpy-1.16.6
     |████████████████████████████████| 15.7 MB 1.7 MB/s eta 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorlayer 2.1.0 requires numpy<1.17,>=1.16, but you have numpy 1.21.4 which is incompatible.
tensorflow 2.4.1 requires numpy~=1.19.2, but you have numpy 1.21.4 which is incompatible.
tensorflow 2.4.1 requires wrapt~=1.12.1, but you have wrapt 1.11.1 which is incompatible.


In [1]:
import numpy as np
import pandas as pd
import math
import time

import plotly.express as px

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.callbacks as cb

from utils import data_tools,config

In [2]:
np.random.seed(2021)
tf.random.set_seed(2021)
cfg=config.read_config("default.cfg")

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

In [4]:
y_train_1hot = keras.utils.to_categorical(y_train, cfg.n_classes)
y_test_1hot = keras.utils.to_categorical(y_test, cfg.n_classes)

plt_x,plt_y=x_test[:cfg.batch_size],y_test[:cfg.batch_size]
train_loader = tf.data.Dataset.from_tensor_slices((x_train,y_train))

In [5]:
meta_train_loader,meta_val_loader,source_class_loader,val_class_loader=data_tools.form_meta_loader(train_loader,cfg)

In [6]:
start_time=time.time()
for ld in source_class_loader:
    for xx,yy in ld.batch(500):
        pass
print(f"{time.time()-start_time:.03f} sec")

8.595 sec


In [7]:
start_time=time.time()
for ld in val_class_loader:
    for xx,yy in ld.batch(500):
        pass
print(f"{time.time()-start_time:.03f} sec")

5.731 sec


In [12]:
## Embedding ##
class Encoder(keras.Model):
    def __init__(self,n_layers=2,latent_features=2,hidden_featrues=64):
        super().__init__()
        self.latent_features=latent_features
        # First Layer
        self.conv_0=keras.layers.Conv2D(hidden_featrues, kernel_size=3, activation="relu",padding='same')
        self.dropout_0=keras.layers.Dropout(0.2)
        # Middle Layers
        self.convs=[keras.layers.Conv2D(hidden_featrues, kernel_size=3, activation="relu",padding='same') for _ in range(n_layers)]
        self.mxpools = [keras.layers.MaxPooling2D(pool_size=2) for _ in range(n_layers)]
        self.dropouts = [keras.layers.Dropout(0.2) for _ in range(n_layers)]
        # Integration Layer(Output Layer)
        WIDTH=2
        self.gap=keras.layers.AveragePooling2D(pool_size=WIDTH,)
        self.flatten=keras.layers.Flatten()
        self.dense=keras.layers.Dense(latent_features)
      
    def call(self,x):
        x=self.dropout_0(self.conv_0(x))
        for conv,mxp,drp in zip(self.convs,self.mxpools,self.dropouts):
            x=drp(mxp(conv(x)))
        x=self.gap(x)
        x=self.flatten(x)
        y=self.dense(x)
        return y

## Special Layers ##
class MatMul:
    def __init__(self,mode="paired"):
        self.mode=mode
        assert mode in ["paired","cross"]
    def __call__(self,q,s):
        if self.mode=="paired":
            return tf.reduce_sum(tf.multiply(q,s),axis=-1,keepdims=True)
        if self.mode=="cross":
            return tf.matmul(q,s,transpose_b=True)
        
class EuclideanDist:
    def __init__(self,mode="paired"):
        self.mode=mode
        assert mode in ["paired","cross"]
        
    def __call__(self,q,s):
        if self.mode=="paired":
            dist=tf.math.reduce_euclidean_norm(q-s,axis=-1,keepdims=True)
        if self.mode=="cross":
            dist=-tf.math.reduce_euclidean_norm(
                tf.stack([q[:,qq:qq+1]-s for qq in range(q.shape[1])],axis=1)
                ,axis=-1,keepdims=False)
            
        return dist

class CosSim:
    def __init__(self,mode="paired"):
        self.mode=mode
        assert mode in ["paired","cross"]
        self.matmul=MatMul(mode)
    def __call__(self,q,s):
        query_n=tf.math.l2_normalize(q, axis=-1)
        support_n=tf.math.l2_normalize(s, axis=-1)
        return self.matmul(query_n,support_n)
    
class CosineLayer(keras.layers.Layer):
    def __init__(self, out_features=10,mode="cross"):
        super().__init__()
        self.out_features = out_features
        self.cos_sim=CosSim(mode)
    def build(self, input_shape):
        self.w = self.add_weight(shape=(self.out_features,input_shape[-1]),
                               initializer='glorot_uniform',
                               trainable=True)
    def call(self, inputs):
        return self.cos_sim(inputs,self.w)


    
## Main Model ##
class ClassModel(keras.Model):
    def __init__(self,encoder,out_features=10,mode="dense"):
        super().__init__()
        assert mode in ["dense","cosmargin","arcmargin"]
        self.encoder=encoder
        self.mode=mode
        self.latent_features=encoder.latent_features
        
        if self.mode=="dense":
            self.output_layer=keras.layers.Dense(out_features,use_bias=False)
            self.loss_fn=FocalLoss(3)
            return
        
        self.output_layer=CosineLayer(out_features)
        if self.mode=="cosmargin":
            self.loss_fn=AddMarginLoss(s=5,m=0.3)
        if self.mode=="arcmargin":
            self.loss_fn=ArcMarginLoss(s=5,m=0.4,easy_margin=False)
        
    def call(self,x,training=False):
        latent=self.encoder(x)
        return self.output_layer(latent)

class SiameseModel(keras.Model):
    # Mode: dist= euclidean distance
    #       cos = cosine distance
    #       matmult= matric multiplication distance
    def __init__(self,encoder,mode="cos"):
        super().__init__()
        assert mode in ["dist","cos","matmul"]
        self.encoder=encoder
        self.mode=mode
        self.latent_features=encoder.latent_features
        if self.mode=="dist":
            self.matric_fn=EuclideanDist("paired")
        if self.mode=="cos":
            self.matric_fn=CosSim("paired")
        if self.mode=="matmul":
            self.matric_fn=MatMul("paired")
        self.loss_fn=ContrastiveLoss(1,mode=mode)
            
    def call(self,x,training=False):
        latent_s=self.encoder(x[:,0])
        latent_q=self.encoder(x[:,1])

        return self.matric_fn(latent_q,latent_s)     
  
    
class PrototypicalNetworks(keras.Model):
    # Mode: dist= euclidean distance
    #       cos = cosine distance
    #       matmult= matric multiplication distance
    def __init__(self,encoder,mode="cos"):
        super().__init__()
        assert mode in ["dist","cos","matmul","cosmargin","arcmargin"]
        self.encoder=encoder
        self.mode=mode
        self.latent_features=encoder.latent_features
        #Metric function#    
        if mode in ["cos","cosmargin","arcmargin"]:
            self.matric_fn=CosSim("cross")
        if self.mode=="dist":
            self.matric_fn=EuclideanDist("cross")
        if self.mode=="matmul":
            self.matric_fn=MatMul("cross")
        #Loss function, including output activation#
        if mode in ["dist","cos","matmul"]:
            self.loss_fn=FocalLoss(3)
        if self.mode=="cosmargin":
            self.loss_fn=AddMarginLoss(s=5,m=0.3)
        if self.mode=="arcmargin":
            self.loss_fn=ArcMarginLoss(s=5,m=0.4,easy_margin=False)  
            
    def call(self,x,training=False):
        # 計算query latent
        latent_q=tf.stack([self.encoder(x[:,WAYS*SHOTS+ii]) for ii in range(WAYS)],axis=1)        
        
        # 計算 prototypes
        latent_s=[self.encoder(x[:,ii]) for ii in range(WAYS*SHOTS)]
        latent_proto=tf.stack([tf.reduce_mean(tf.stack(latent_s[ww*SHOTS:(ww+1)*SHOTS],axis=-1),axis=-1)
                     for ww in range(WAYS)],axis=1)
        # 計算metric
        return self.matric_fn(latent_q,latent_proto)

In [13]:
enc=Encoder(n_layers=2,latent_features=2,hidden_featrues=64)
model=PrototypicalNetworks(enc,mode="arcmargin")# {"dist","cos","matmul","cosmargin","arcmargin"}

In [14]:
# loss_fn=ArcMarginLoss(s=5,m=0.4,easy_margin=False)
loss_fn=model.loss_fn
opt=keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=loss_fn, optimizer=opt, metrics=[keras.metrics.CategoricalAccuracy()])

In [15]:
## Recording Callbacks
class LogLatent(keras.callbacks.Callback):
    def __init__(self,encoder,test_x,test_y,skip_iters=100):
        self.df=self.df=pd.DataFrame()
        self.encoder=encoder
        self.plt_x=test_x
        self.plt_y=test_y
        self.itr=0
        self.skip_iters=skip_iters
    def on_batch_end(self,batch, logs={}):
        if self.itr%self.skip_iters==self.skip_iters-1:
            latent=self.encoder(self.plt_x)
            df_new=pd.DataFrame()
            df_new["f1"]=latent[:,0]
            df_new["f2"]=latent[:,1]
            df_new["arc"]=0.5
            df_new["iterations"]=np.repeat(self.itr,len(latent))

            df_new["label"]=self.plt_y
            self.df=self.df.append(df_new)
        self.itr+=1
    def normalize(self):
        # Normalize latent by maximum value
        self.mx_norm=np.linalg.norm(self.df[["f1","f2"]].values[-len(self.plt_y):],axis=1).max()
        self.df[["f1",'f2']]=self.df[["f1",'f2']].apply(lambda x: x/self.mx_norm)

        # Project latents into arc
        df=self.df.copy()
        norm=np.linalg.norm(df[["f1","f2"]].values,axis=1,keepdims=True)
        df[["f1",'f2']]=df[["f1",'f2']].values/norm
        df["arc"]=df["arc"].map({0.5:0.9},na_action=None)
        
        self.df=self.df.append(df)

In [16]:

train_loader = train_meta_loader.shuffle(1000).cache().prefetch(meta_batch_size).batch(meta_batch_size)

val_loader = train_meta_loader.shuffle(1000).cache().prefetch(meta_batch_size*4).batch(meta_batch_size*4)


In [17]:
lg=LogLatent(enc,plt_x,plt_y)
reduce_lr = cb.ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=5, min_lr=1e-8, verbose=1)
earlystop=cb.EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True, verbose=1)
try:
    model.fit(train_loader, 
              epochs=800,
              validation_data=val_loader,
              callbacks=[lg,reduce_lr,earlystop])
except KeyboardInterrupt:
    print("KeyboardInterrupt")

Epoch 1/800
19/19 [==============================] - 1358s 38s/step - loss: 1.8769 - categorical_accuracy: 0.3510 - val_loss: 1.2444 - val_categorical_accuracy: 0.5200
Epoch 2/800
19/19 [==============================] - 1s 56ms/step - loss: 1.2846 - categorical_accuracy: 0.4491 - val_loss: 1.2037 - val_categorical_accuracy: 0.5417
Epoch 3/800
19/19 [==============================] - 1s 53ms/step - loss: 1.2103 - categorical_accuracy: 0.5112 - val_loss: 0.9320 - val_categorical_accuracy: 0.5777
Epoch 4/800
19/19 [==============================] - 1s 52ms/step - loss: 1.0122 - categorical_accuracy: 0.5252 - val_loss: 0.8174 - val_categorical_accuracy: 0.6133
Epoch 5/800
19/19 [==============================] - 1s 53ms/step - loss: 0.8650 - categorical_accuracy: 0.5997 - val_loss: 0.7139 - val_categorical_accuracy: 0.6610
Epoch 6/800
19/19 [==============================] - 1s 56ms/step - loss: 0.7480 - categorical_accuracy: 0.6285 - val_loss: 0.5893 - val_categorical_accuracy: 0.7233
Ep

In [18]:
lg.normalize()

In [ ]:
fig=px.scatter(lg.df, x="f1", y="f2", animation_frame="iterations", color="label",opacity=lg.df.arc,
                range_x=[-1.5,1.5],
              range_y=[-1.5,1.5])
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
)

In [20]:
fig.write_html("history/proto_arcmargin_tf.html")

In [30]:
model_inference=ClassModel(enc,mode="arcmargin")# {"dist","cos","matmul","cosmargin","arcmargin"}
opt=keras.optimizers.Adam(learning_rate=1e-3)
model_inference.compile(loss=loss_fn, optimizer=opt, metrics=[keras.metrics.CategoricalAccuracy()])
model_inference.build((None,28,28,1))
def take_prototype(class_loader,encoder):
    latent=[]
    for loader in class_loader:
        x,y=next(iter(loader.batch(20)))
        latent.append(encoder(x))
    return tf.reduce_mean(tf.stack(latent,axis=0),axis=1)
proto=take_prototype(all_class_list,enc)
model_inference.output_layer.weights[0].assign(proto)

<tf.Variable 'UnreadVariable' shape=(10, 2) dtype=float32, numpy=
array([[ 0.18704899,  0.02250218],
       [ 0.12121026, -0.10678957],
       [ 0.02535258,  0.24400404],
       [-0.15941562,  0.19275862],
       [-0.022591  , -0.22673993],
       [-0.22677712,  0.07567718],
       [ 0.15992527,  0.15428177],
       [-0.11859933, -0.12187711],
       [-0.17760316,  0.10250525],
       [-0.16005751, -0.1558148 ]], dtype=float32)>

In [31]:
model_inference.evaluate(x_test,y_test_1hot)

313/313 [==============================] - 1s 2ms/step - loss: 0.7114 - categorical_accuracy: 0.7250


[0.654492974281311, 0.7486000061035156]

In [25]:


train_finetune_loader = tf.data.Dataset.from_tensor_slices((x_train,y_train_1hot)).cache().shuffle(len(x_train)//2).batch(batch_size)
val_finetune_loader = tf.data.Dataset.from_tensor_slices((x_val,y_val_1hot)).cache().shuffle(len(x_val)//2).batch(batch_size*2)

In [32]:
lg=LogLatent(enc,plt_x,plt_y)
try:
    model_inference.fit(train_finetune_loader, 
              epochs=epochs,
              validation_data=val_finetune_loader,
              callbacks=[lg])
except KeyboardInterrupt:
    print("KeyboardInterrupt")

Epoch 1/10
375/375 [==============================] - 5s 10ms/step - loss: 0.6885 - categorical_accuracy: 0.7538 - val_loss: 0.5536 - val_categorical_accuracy: 0.8406
Epoch 2/10
375/375 [==============================] - 4s 10ms/step - loss: 0.5600 - categorical_accuracy: 0.8357 - val_loss: 0.5089 - val_categorical_accuracy: 0.8633
Epoch 3/10
375/375 [==============================] - 4s 10ms/step - loss: 0.5199 - categorical_accuracy: 0.8613 - val_loss: 0.4311 - val_categorical_accuracy: 0.9054
Epoch 4/10
375/375 [==============================] - 4s 10ms/step - loss: 0.4887 - categorical_accuracy: 0.8738 - val_loss: 0.4166 - val_categorical_accuracy: 0.9107
Epoch 5/10
375/375 [==============================] - 4s 10ms/step - loss: 0.4512 - categorical_accuracy: 0.8907 - val_loss: 0.3956 - val_categorical_accuracy: 0.9212
Epoch 6/10
375/375 [==============================] - 4s 10ms/step - loss: 0.4526 - categorical_accuracy: 0.8913 - val_loss: 0.4266 - val_categorical_accuracy: 0.910

In [33]:
lg.normalize()

In [ ]:
fig=px.scatter(lg.df, x="f1", y="f2", animation_frame="iterations", color="label",opacity=lg.df.arc,
                range_x=[-1.5,1.5],
              range_y=[-1.5,1.5])
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
)

In [37]:
model_inference.evaluate(x_test,y_test_1hot)

313/313 [==============================] - 1s 2ms/step - loss: 0.3438 - categorical_accuracy: 0.9372


[0.3437580466270447, 0.9372000098228455]

In [38]:
fig.write_html("history/proto_baseline_arcmargin_tf.html")